### Vader Sentiment

* compound: [-1, 1]
  * positive sentiment: compound score >= 0.05
    * [0.05, 1] -> 4, 5
    * [0.05, 0.525] ->4
    * [0.525, 1] -> 5
  * neutral sentiment: (compound score > -0.05) and (compound score < 0.05)
    * (-0.05, 0.05) -> 3
  * negative sentiment: compound score <= -0.05
    * [-1, -0.05] -> 1, 2
    * [-1, -0.525] -> 1
    * [-0.525, -0.05] -> 2

### TextBlob

* polarity: [-1, 1]
* subjectivity: [0, 1]
    
### Stanford CoreNLP
* use deeplearning algorithm to handle the order of words
* sentimentValue= 4 , sentiment= Verypositive
* sentimentValue= 3 , sentiment= Positive
* sentimentValue= 2 , sentiment= Neutral
* sentimentValue= 1 , sentiment= Negative
* sentimentValue= 0 , sentiment= Verynegative
    
### LSTM
* test dataset: 
  * took > 1 hour
  * train : test = 2 :1
  * acc: 71% of 1500 validation size on testing dataset, 
* train dataset:  
  * took 44725.2392093s =12.4 hours
  * train : test = 2 :1
  * acc: 79% on whole test dataset by model.evaluate()
  * acc: 100% on whole test dataset by labels compare
    * predict 31785 samples, took 1002s,  31.5ms/sample



In [32]:
sentence = "It's not a course... this is a very short general introduction to 3D Printing"


In [33]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyser = SentimentIntensityAnalyzer()
score = analyser.polarity_scores(sentence)
print("{:-<40} {}".format(sentence, str(score)))
score['compound']

It's not a course... this is a very short general introduction to 3D Printing {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


0.0

In [34]:
from textblob import TextBlob

testimonial = TextBlob(sentence)
print(testimonial.sentiment.polarity)

0.02500000000000001


In [35]:

from pycorenlp import StanfordCoreNLP

nlp = StanfordCoreNLP('http://localhost:9000')
output = nlp.annotate(sentence, properties={
            'annotators': 'sentiment',
            'outputFormat': 'json'
        })
print(output['sentences'][0]['sentimentValue'])
print(output['sentences'][0]['sentiment'])
print(output['sentences'][0]['sentimentDistribution'])
print(output['sentences'][0]['sentimentTree'])

1
Negative
[0.13210145953291, 0.65314998335187, 0.19261037575247, 0.01472903402969, 0.00740914733306]
(ROOT|sentiment=1|prob=0.653
  (@S|sentiment=2|prob=0.590
    (S|sentiment=2|prob=0.509 (NP|sentiment=2|prob=0.996 It)
      (VP|sentiment=2|prob=0.583
        (@VP|sentiment=2|prob=0.857 (VBZ|sentiment=2|prob=0.994 's) (RB|sentiment=1|prob=0.974 not))
        (NP|sentiment=2|prob=0.964 (DT|sentiment=2|prob=0.990 a) (NN|sentiment=2|prob=0.993 course))))
    (:|sentiment=2|prob=0.997 ...))
  (S|sentiment=1|prob=0.461 (NP|sentiment=2|prob=0.998 this)
    (VP|sentiment=1|prob=0.439 (VBZ|sentiment=2|prob=0.989 is)
      (NP|sentiment=2|prob=0.618
        (NP|sentiment=2|prob=0.456 (DT|sentiment=2|prob=0.990 a)
          (@NP|sentiment=2|prob=0.569
            (ADJP|sentiment=2|prob=0.706 (RB|sentiment=2|prob=0.995 very) (JJ|sentiment=2|prob=0.988 short))
            (@NP|sentiment=2|prob=0.961 (JJ|sentiment=2|prob=0.993 general) (NN|sentiment=2|prob=0.990 introduction))))
        (PP|senti

In [69]:
from pattern.en import sentiment

scores = sentiment(sentence)
print(scores[0])

0.02500000000000001


In [ ]:
### Deep Learning Play

In [77]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

data = pd.read_csv('../100k-courseras-course-reviews-dataset/reviews_train.csv')
# Keeping only the neccessary columns
data = data[['Review','Label']]

data['Review'] = data['Review'].apply(lambda x: x.lower())
data['Review'] = data['Review'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['Label'] == 1].size)
print(data[ data['Label'] == 2].size)
print(data[ data['Label'] == 3].size)
print(data[ data['Label'] == 4].size)
print(data[ data['Label'] == 5].size)

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['Review'].values)
X = tokenizer.texts_to_sequences(data['Review'].values)
X = pad_sequences(X)


4244
3844
8854
32156
143534


In [78]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(5,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 1148, 128)         256000    
_________________________________________________________________
spatial_dropout1d_5 (Spatial (None, 1148, 128)         0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 985       
Total params: 511,785
Trainable params: 511,785
Non-trainable params: 0
_________________________________________________________________
None


In [79]:
Y = pd.get_dummies(data['Label']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(64531, 1148) (64531, 5)
(31785, 1148) (31785, 5)


In [65]:
import timeit

start = timeit.default_timer()


batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

model.save('LSTM_train_v0.1.h5') 
stop = timeit.default_timer()
print('Time: ', stop - start)

Epoch 1/7
 - 6236s - loss: 0.6490 - acc: 0.7657
Epoch 2/7
 - 6282s - loss: 0.5629 - acc: 0.7872
Epoch 3/7
 - 6296s - loss: 0.5346 - acc: 0.7980
Epoch 4/7
 - 6336s - loss: 0.5202 - acc: 0.8042
Epoch 5/7
 - 6297s - loss: 0.5055 - acc: 0.8081
Epoch 6/7
 - 6521s - loss: 0.4920 - acc: 0.8148
Epoch 7/7
 - 6757s - loss: 0.4793 - acc: 0.8188
Time:  44725.2392093


In [81]:
from keras.models import load_model

print(X_test.shape,Y_test.shape)
model = load_model('LSTM_train_v0.1.h5')
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

(31785, 1148) (31785, 5)
score: 0.57
acc: 0.79


In [54]:
# first lstm model on test data set, got score: 0.80, acc: 0.71
# model.save('LSTM_test_v0.1.h5') 
# first lstm model on train data set, got score: 0.57, acc: 0.79
# model.save('LSTM_train_v0.1.h5') 

In [101]:
start = timeit.default_timer()

model = load_model('LSTM_train_v0.1.h5')
results = model.predict(X_test, batch_size=batch_size, verbose=2, steps=None)
predict_labels = [np.argmax(x)+1 for x in results]
actual_labels = [np.argmax(x)+1 for x in Y_test]
corrects = [ 1 if predict_labels[i] == actual_labels[i] else 0 for i in range(0, len(labels))]
acc = sum(corrects)/len(corrects)
print('label accuracy = ', acc)

stop = timeit.default_timer()
print('Predicting tooks(s): ', stop - start)


accuracy =  1.0
Predicting tooks(s):  1002.3016105999995


In [100]:
labels = [np.argmax(x)+1 for x in results]
actual_labels = [np.argmax(x)+1 for x in Y_test[0:5]]
corrects = [ 1 if labels[i] == actual_labels[i] else 0 for i in range(0, len(labels))]
print(sum(corrects)/len(corrects))

1.0


In [137]:
############# preprocess for LSTM

def get_tokenizer(dataFile, max_fatures):
    data = pd.read_csv(dataFile)
    data = data[['Review','Label']]

    data['Review'] = data['Review'].apply(lambda x: x.lower())
    data['Review'] = data['Review'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
    
    tokenizer = Tokenizer(num_words=max_fatures, split=' ')
    tokenizer.fit_on_texts(data['Review'].values)
    X = tokenizer.texts_to_sequences(data['Review'].values)
    X = pad_sequences(X)
    print(data['Review'][0:3])
    print(X.shape)
    print(X[0:3])
    Y = pd.get_dummies(data['Label']).values
    
    return tokenizer, X.shape[1]

tokenizer, feature_len = get_tokenizer('../100k-courseras-course-reviews-dataset/reviews.csv', max_fatures = 2000)

def preprocess_review(review):
    review = re.sub('[^a-zA-z0-9\s]','',review.lower())
    sequence = tokenizer.texts_to_sequences([review])
    feature_vec = pad_sequences(sequence, maxlen=feature_len)
    return feature_vec

model = load_model('LSTM_v0.1.h5')
labels = ['VeryNegative', 'Negative', 'Netural', 'Positive', 'VeryPositive']

def label_review(review):
    feature_vec=preprocess_review(review)
    predict_values = model.predict(feature_vec, batch_size=32, verbose=2, steps=None)
    label_index = np.argmax(predict_values)
    
    
#     print('"', review, '"', ' IS ', labels[label_index])
    
    return label_index + 1

0                                 good and interesting
1    this class is very helpful to me currently im ...
2    likeprof and tas are helpful and the discussio...
Name: Review, dtype: object
(107018, 1148)
[[   0    0    0 ...   16    2   39]
 [   0    0    0 ...  102  460   78]
 [   0    0    0 ... 1045   31  115]]


In [138]:


data = pd.read_csv('../100k-courseras-course-reviews-dataset/reviews_test.csv')
data = data[['Review','Label']]
data['Predict_Label'] = data['Review'].apply(lambda x: label_review(x))

corrects = data[ data['Label'] == data['Predict_Label']]
print(corrects.shape)
print('acc = ', )

(8527, 3)
acc = 
